In [1]:
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.utils.data as torchdata
import torchvision.models as models
import gc
import time
from sklearn.metrics import precision_recall_fscore_support,roc_auc_score, accuracy_score
from sklearn.metrics import precision_score,f1_score
import albumentations as albu
from sklearn.metrics import confusion_matrix

In [2]:
#Loading the masked images

train = np.load('fold1_class_echo_train.npy')
valid = np.load('fold1_class_echo_valid.npy')
test = np.load('fold1_class_echo_test.npy')

In [3]:
train = train.reshape(-1,224,224,3).transpose(0,3,1,2)
valid = valid.reshape(-1,224,224,3).transpose(0,3,1,2)
test = test.reshape(-1,224,224,3).transpose(0,3,1,2)

In [4]:
#Loading the ground truthe labels for the segments

train_labels = np.load('Fold1_echo_lab_train.npy')
test_labels = np.load('Fold1_echo_lab_test.npy')
valid_labels = np.load('Fold1_echo_lab_valid.npy')

train_labels = train_labels#,1)
test_labels = test_labels#,1)
valid_labels = valid_labels#,1)

In [5]:
#list of albumentations transformations we used

transform = [
    albu.augmentations.Rotate(limit=90,p=0.9),
    albu.augmentations.ElasticTransform(p=0.9),
    albu.IAAAdditiveGaussianNoise(p=0.9),
    albu.OneOf([
        albu.CLAHE(p=1),
        albu.RandomBrightness(p=1),
        albu.RandomGamma(p=1),
    ],p=0.9),
    albu.OneOf([
        albu.IAASharpen(p=1),
        albu.Blur(blur_limit=3, p=1),
        albu.MotionBlur(blur_limit=3, p=1),
    ],p=0.9)
 
    ]

augment = albu.Compose(transform)

In [6]:
#Conversion to tensors and float 32

train_data = torch.tensor(train, dtype=torch.float32)
train_lab = torch.tensor(train_labels, dtype=torch.float32)
valid_data = torch.tensor(valid, dtype=torch.float32)
valid_lab = torch.tensor(valid_labels, dtype=torch.float32)
test_data = torch.tensor(test, dtype=torch.float32)
test_lab = torch.tensor(test_labels, dtype=torch.float32)

In [7]:
#Concatenating input data and labels

training = []
for i in range(len(train_data)):
    training.append([train_data[i], train_lab[i]])
    
validation = []
for i in range(len(valid_data)):
    validation.append([valid_data[i], valid_lab[i]])
    
testing = []
for i in range(len(test_data)):
    testing.append([test_data[i], test_lab[i]])

In [8]:
#Storing to DataLoader
train_batch = 20
batch = 10

train_dl =  DataLoader(training, batch,shuffle=True)
valid_dl = DataLoader(validation, batch)
test_dl = DataLoader(testing, batch)

In [9]:
#Wrapping models and loaders to gpu
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [10]:
#Storing to gpu
device = get_default_device()
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(test_dl, device)
test_dl = DeviceDataLoader(test_dl, device)

In [11]:
#setting hyperparameters
epochs = 25
alpha = 0.001

In [12]:
#Loading resnest50 architecture
from torchvision.models import resnet50, resnet101
from resnest.torch import resnest50

class CNNLSTM(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNLSTM, self).__init__()
        self.resnest = resnet50(pretrained=True)
        self.resnest.fc = nn.Sequential(nn.Linear(self.resnest.fc.in_features, 300))
        self.lstm = nn.LSTM(input_size=300, hidden_size=256, num_layers=3)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)
       
    def forward(self, x_3d):
        hidden = None
        for t in range(x_3d.size(1)):
            with torch.no_grad():
                x = self.resnest(x_3d[:, -1, :, :, :])  
            out, hidden = self.lstm(x.unsqueeze(0), hidden)         

        x = self.fc1(out[-1, :, :])
        x = F.relu(x)
        x = self.fc2(x)
        return x

model = CNNLSTM()

In [13]:
model = resnest50(pretrained=True)

#Fully connected layer
model.fc = nn.Sequential(nn.Linear(2048, 6))#,
                        #nn.LSTM(input_size=300, hidden_size=256, num_layers=3),
                        #nn.Linear(256,128),
                        #nn.Linear(128,6))

In [14]:
#Storing the results of the runs

result_arr = torch.zeros((epochs,3))
best = 0

In [15]:
to_device(model, device)
optimizer = optim.Adam(model.parameters(), lr=alpha)
#Since this is a multi-label classification, BCE with Logits loss is used
loss_function = nn.BCEWithLogitsLoss()
loss_function = loss_function.cuda()

start_time = time.time()

for epoch in range(epochs):
    
    val_out = torch.zeros(0).cuda()
    model.train()
    t_count = 0
    t_loss = 0
    t_acc = 0
    
    for data,target in train_dl:
        arr = []
        
        for i in range(data.shape[0]):
            
            np_img = data[i].cpu().numpy()
            np_img = np_img.transpose(1,2,0)
            sample = augment(image=np_img.astype('uint8'))
            img = sample['image']
            arr.append(img)
            
        data = np.array(arr)
        data = data.transpose(0,3,1,2)
        data = torch.tensor(data, dtype=torch.float32)
        data = data.cuda()
        
        #data = torch.vstack((data,data))
        #target = torch.vstack((target,target))
        
        t_count += 1
        
        optimizer.zero_grad()
        output = model(data)
        
        preds = torch.sigmoid(output)
        preds = torch.round(preds)

        loss = loss_function(output, target)
        
        #Computing a running total of loss and accuracy
        t_loss += loss.item()
        acc = torch.sum(preds==target).item()/(target.shape[0]*target.shape[1])
        t_acc += acc
        

        loss.backward()
        optimizer.step()
    
    train_accuracy = t_acc/t_count
    train_loss = t_loss/t_count
    
    print('Epoch: ', epoch, 'Training Loss: {:.4f}'.format(train_loss)\
          ,'Training Accuracy: {:.4f}'.format(train_accuracy))
    
    model.eval()
    v_count = 0
    v_loss = 0
    v_acc = 0
    val_out = torch.zeros(0).cuda()

    with torch.no_grad():

        for val_data,val_target in valid_dl:
            v_count += 1
            val_output = model(val_data)

            val_preds = torch.sigmoid(val_output)
            val_preds = torch.round(val_preds)

            val_out = torch.cat((val_out,val_preds),axis=0)

            val_loss = loss_function(val_output, val_target)

            v_loss += val_loss.item()
            val_acc = torch.sum(val_preds==val_target).item()/(val_target.shape[0]*val_target.shape[1])
            v_acc += val_acc

        val_accuracy = v_acc/v_count
        val_loss = v_loss/v_count

        print('Epoch: ', epoch, ' Validation Loss: {:.4f}'.format(val_loss)\
              ,'Validation Accuracy: {:.4f}'.format(val_accuracy))

        #Saving the best model
        if val_accuracy > best:
            best = val_accuracy
            torch.save(model, './valclass.pth')
    
    #Saving the best model
    #if train_accuracy > best:
        #best = train_accuracy
        #torch.save(model, './trialclass.pth')

    result_arr[epoch,0] = epoch
    result_arr[epoch,1] = train_loss
    result_arr[epoch,2] = train_accuracy

end_time = time.time()
print('Processing Time: ',end_time-start_time)

Epoch:  0 Training Loss: 0.6175 Training Accuracy: 0.6823
Epoch:  0  Validation Loss: 0.4911 Validation Accuracy: 0.7701
Epoch:  1 Training Loss: 0.5980 Training Accuracy: 0.6927
Epoch:  1  Validation Loss: 0.4905 Validation Accuracy: 0.7708
Epoch:  2 Training Loss: 0.5929 Training Accuracy: 0.6914
Epoch:  2  Validation Loss: 0.5161 Validation Accuracy: 0.7502
Epoch:  3 Training Loss: 0.5867 Training Accuracy: 0.6977
Epoch:  3  Validation Loss: 0.4748 Validation Accuracy: 0.7681
Epoch:  4 Training Loss: 0.5952 Training Accuracy: 0.6900
Epoch:  4  Validation Loss: 0.4964 Validation Accuracy: 0.7403
Epoch:  5 Training Loss: 0.5871 Training Accuracy: 0.6965
Epoch:  5  Validation Loss: 0.4880 Validation Accuracy: 0.7438
Epoch:  6 Training Loss: 0.5836 Training Accuracy: 0.6963
Epoch:  6  Validation Loss: 0.5005 Validation Accuracy: 0.7771
Epoch:  7 Training Loss: 0.5813 Training Accuracy: 0.6993
Epoch:  7  Validation Loss: 0.5323 Validation Accuracy: 0.7062
Epoch:  8 Training Loss: 0.5842 

In [16]:
best_model = torch.load('./valclass.pth')

In [17]:
model.eval()
t_count = 0
t_loss = 0
t_acc = 0
test_out = torch.zeros(0).cuda()
        
with torch.no_grad():
        
    for test_data,test_target in test_dl:
        t_count += 1
        test_output = model(test_data)
            
        test_preds = torch.sigmoid(test_output)
        test_preds = torch.round(test_preds)
        
        test_out = torch.cat((test_out,test_preds),axis=0)
        
true_data = test_labels.reshape(-1)
pred_data = test_out.reshape(-1).cpu()

#Accuracy scores
score_acc = accuracy_score(true_data,pred_data)

#Precision scores
score_prf = precision_recall_fscore_support(true_data, pred_data, average='binary')

print('Accuracy: ',score_acc)
print('Scores: ',score_prf)

tn, fp, fn, tp = confusion_matrix(true_data,pred_data).ravel()
specificity = tn/(tn+fp)
specificity

Accuracy:  0.7566807313642757
Scores:  (0.404, 0.3389261744966443, 0.36861313868613144, None)


0.8674377224199288

In [18]:
best_model.eval()
t_count = 0
t_loss = 0
t_acc = 0
test_out = torch.zeros(0).cuda()
        
with torch.no_grad():
        
    for test_data,test_target in test_dl:
        t_count += 1
        test_output = best_model(test_data)
            
        test_preds = torch.sigmoid(test_output)
        test_preds = torch.round(test_preds)
        
        test_out = torch.cat((test_out,test_preds),axis=0)

In [19]:
true_data = test_labels.reshape(-1)
pred_data = test_out.reshape(-1).cpu()

In [20]:
#Accuracy scores
score_acc = accuracy_score(true_data,pred_data)

In [21]:
#Precision scores
score_prf = precision_recall_fscore_support(true_data, pred_data, average='binary')

In [22]:
print('Accuracy: ',score_acc)
print('Scores: ',score_prf)

Accuracy:  0.8129395218002813
Scores:  (0.6095890410958904, 0.2986577181208054, 0.4009009009009009, None)


In [23]:
tn, fp, fn, tp = confusion_matrix(true_data,pred_data).ravel()
specificity = tn/(tn+fp)
specificity

0.949288256227758